## Precomputed_annotation_format
The purpose of this notebook is to show how to make a precomputed annotation layer from a list of 3d points.
Refer to this document: https://github.com/google/neuroglancer/blob/master/src/neuroglancer/datasource/precomputed/annotations.md for general structure (e.g. if you need another type of annotation than points)

In [1]:
import numpy as np
import os
import csv
import struct
import json
from cloudvolume import CloudVolume
import matplotlib.pyplot as plt
import neuroglancer
%matplotlib inline

In [2]:
# get the raw-space cells file and load it in
animal_id = 4
pth=os.path.join('/jukebox/wang/Jess/lightsheet_output',
        '201904_ymaze_cfos','processed',f'an{animal_id}','clearmap_cluster_output',
        'cells.npy')
converted_points = np.load(pth)

In [3]:
converted_points

array([[ 459, 1398,   50],
       [ 459, 1443,   50],
       [ 462, 1412,   49],
       ...,
       [1546, 1242,  569],
       [1547, 1316,  570],
       [1646, 1328,  574]])

In [4]:
# Here is a test to show which method is best to use for making the annotation file. 
# Second method is MUCH faster -- use that one!

In [10]:
np.random.shuffle(converted_points[0:100])
converted_points[0:100]

array([[ 495, 1289,   47],
       [ 489, 1473,   45],
       [ 483, 1453,   49],
       [ 467, 1409,   50],
       [ 497, 1490,   46],
       [ 504, 1293,   45],
       [ 459, 1398,   50],
       [ 504, 1297,   45],
       [ 462, 1412,   49],
       [ 478, 1310,   50],
       [ 505, 1421,   46],
       [ 501, 1244,   50],
       [ 506, 1243,   50],
       [ 482, 1298,   50],
       [ 493, 1503,   47],
       [ 509, 1464,   41],
       [ 508, 1447,   47],
       [ 486, 1406,   45],
       [ 507, 1534,   46],
       [ 470, 1441,   50],
       [ 475, 1447,   46],
       [ 506, 1387,   44],
       [ 508, 1494,   47],
       [ 510, 1232,   50],
       [ 484, 1350,   47],
       [ 500, 1526,   50],
       [ 505, 1565,   50],
       [ 503, 1449,   42],
       [ 477, 1454,   48],
       [ 484, 1344,   46],
       [ 502, 1472,   44],
       [ 506, 1532,   49],
       [ 504, 1396,   42],
       [ 475, 1336,   48],
       [ 497, 1345,   44],
       [ 507, 1226,   50],
       [ 480, 1374,   46],
 

In [6]:
np.random.shuffle(converted_points[0:100])

In [45]:
####################################################
### SLOW!! - DO NOT USE - JUST FOR BENCHMARKING! ###
####################################################
%%time
filename = '/home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0'

coordinates = converted_points # array like [[x1,y1,z1],[x2,y2,z2],...]
total_count = len(coordinates)
with open(filename,'wb') as outfile:
    buf = struct.pack('<Q',total_count)
    for (x,y,z) in coordinates:
        pt_buf = struct.pack('<3f',x,y,z)
        buf+=pt_buf
    id_buf = struct.pack('<%sQ' % len(coordinates), *range(len(coordinates)))
    buf+=id_buf
    outfile.write(buf)
print(f"wrote {filename}")

wrote /home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0
CPU times: user 3.98 s, sys: 16 ms, total: 3.99 s
Wall time: 3.97 s


In [47]:
%%time
filename = '/home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0'

coordinates = converted_points
total_count = len(coordinates)
with open(filename,'wb') as outfile:
    buf = struct.pack('<Q',total_count)
    pt_buf = b''.join(struct.pack('<3f',x,y,z) for (x,y,z) in coordinates)
    buf += pt_buf
    id_buf = struct.pack('<%sQ' % len(coordinates), *range(len(coordinates)))
    buf += id_buf
    outfile.write(buf)
print(f"wrote {filename}")

wrote /home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawannotations_an4/spatial0/0_0_0
CPU times: user 584 ms, sys: 40 ms, total: 624 ms
Wall time: 622 ms
